In [43]:
import os
import re
import numpy as np
from scipy import ndimage

import sys
import pickle
import json
import tables

import importlib.util #import from superordinate folder
spec = importlib.util.spec_from_file_location("helper", 
                                              str(re.search("(.*/)",os.getcwd()).groups(0)[0])+"helper.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

# Decode motion using decode_motion

In [3]:
videos = ["RPS_C_13_C3.wmv", "RPS_FP_13_C3.wmv", "RPS_PS_13_C3.wmv"]
identifiers = [re.search("(.*)\.wmv",x).groups()[0] for x in videos]# cut off ending
outfiles = [x+".hdf5" for x in identifiers]

In [ ]:
for vid, outfile in zip(videos,outfiles):
    ! ./decode_motion {vid} {outfile+"new"}

Input #0, asf, from 'RPS_C_13_C3.wmv':
  Metadata:
    WMFSDKNeeded    : 0.0.0.0000
    WMFSDKVersion   : 12.0.16299.15
    IsVBR           : 0
  Duration: 00:11:59.36, start: 0.000000, bitrate: 1774 kb/s
    Stream #0:0(eng): Video: h264 (Main) (H264 / 0x34363248), yuvj420p(pc, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 50000 kb/s, 25 fps, 25 tbr, 1k tbn, 2k tbc
    Stream #0:1(eng): Audio: aac (LC) ([255][0][0][0] / 0x00FF), 16000 Hz, mono, fltp, 64 kb/s

**************************************************************************************
*       Tool : Dense Flow Decoder                                                    *
*     Author : Michael Gerstenberger (see copyright for version history and authors) *
*  Used Libs : FFmpeg, HDF5                                                          *
*Description : A tool to extract motion vectors from H264 videos and save them as    *
             : a tensor of size n_frames x width x height x 2 (for dx and dy) in     *
        

# Compute and save 1d movement signal for all ROIs for both participants

In [3]:
directory = os.getcwd() + "/"
hdf5er = np.array([x for x in os.listdir(directory)])[np.array([".hdf5" in x for x in os.listdir(directory)])]
outfolder = directory

In [4]:
#For all hdf5, for all head hands and body jsons compute weighted histograms if input was present
one_d = {}
two_d = {}
mot = {}
for file in hdf5er:
    root = re.search("(.*)\.",file).groups()[0]
    identifier = [root+"_left_head",root+"_left_hands",root+"_left_body",
                 root+"_right_head",root+"_right_hands",root+"_right_body"]
    jsons = [root+"_left_head.json",root+"_left_hands.json",root+"_left_body.json",
            root+"_right_head.json",root+"_right_hands.json",root+"_right_body.json"]
    for i,j in zip(identifier,jsons):
        try:
            with open(directory+j, encoding='utf-8') as fh:
                roi = json.load(fh)
                h264 = tables.open_file(directory+file, mode='r')
                h264 = h264.root.motion_tensor
                mothistmap = helper.weighted_histograms(h264,roi=roi)
                mot[i] = mothistmap#append mothistmap
                
                hist = ndimage.gaussian_filter(mothistmap,10)
                amount = np.sum(hist,axis=0)#1d signal
                one_d[i] = amount
                
                #Avoid multikollinearity that arises from smoothing
                left = np.sum(mothistmap[0:5,:],axis=0)+np.sum(mothistmap[31:,:],axis=0)
                down = np.sum(mothistmap[5:14,:],axis=0)
                right = np.sum(mothistmap[14:23,:],axis=0)
                up = np.sum(mothistmap[23:31,:],axis=0)
                two_d[i]= np.array([up,left,down,right])
        except Exception as e:
            print(e)

0.0
0.0055617352614015575
0.011123470522803115
0.01668520578420467
0.02224694104560623
0.027808676307007785
0.03337041156840934
0.0389321468298109
0.04449388209121246
0.05005561735261402
0.05561735261401557
0.06117908787541713
0.06674082313681869
0.07230255839822025
0.0778642936596218
0.08342602892102335
0.08898776418242492
0.09454949944382647
0.10011123470522804
0.10567296996662959
0.11123470522803114
0.1167964404894327
0.12235817575083426
0.12791991101223582
0.13348164627363737
0.13904338153503892
0.1446051167964405
0.15016685205784205
0.1557285873192436
0.16129032258064516
0.1668520578420467
0.1724137931034483
0.17797552836484984
0.1835372636262514
0.18909899888765294
0.1946607341490545
0.20022246941045607
0.20578420467185762
0.21134593993325917
0.21690767519466073
0.22246941045606228
0.22803114571746386
0.2335928809788654
0.23915461624026696
0.2447163515016685
0.25027808676307006
0.25583982202447164
0.26140155728587317
0.26696329254727474
0.2725250278086763
0.27808676307007785
0.28

0.41713014460511677
0.42269187986651835
0.42825361512791993
0.43381535038932145
0.43937708565072303
0.44493882091212456
0.45050055617352613
0.4560622914349277
0.46162402669632924
0.4671857619577308
0.4727474972191324
0.4783092324805339
0.4838709677419355
0.489432703003337
0.4949944382647386
0.5005561735261401
0.5061179087875417
0.5116796440489433
0.5172413793103449
0.5228031145717463
0.5283648498331479
0.5339265850945495
0.5394883203559511
0.5450500556173526
0.5506117908787542
0.5561735261401557
0.5617352614015573
0.5672969966629589
0.5728587319243604
0.578420467185762
0.5839822024471635
0.5895439377085651
0.5951056729699666
0.6006674082313682
0.6062291434927698
0.6117908787541713
0.6173526140155728
0.6229143492769744
0.628476084538376
0.6340378197997776
0.639599555061179
0.6451612903225806
0.6507230255839822
0.6562847608453838
0.6618464961067854
0.6674082313681868
0.6729699666295884
0.67853170189099
0.6840934371523916
0.6896551724137931
0.6952169076751946
0.7007786429365962
0.70634037

0.8453837597330367
0.8509454949944383
0.8565072302558399
0.8620689655172413
0.8676307007786429
0.8731924360400445
0.8787541713014461
0.8843159065628476
0.8898776418242491
0.8954393770856507
0.9010011123470523
0.9065628476084538
0.9121245828698554
0.917686318131257
0.9232480533926585
0.92880978865406
0.9343715239154616
0.9399332591768632
0.9454949944382648
0.9510567296996663
0.9566184649610678
0.9621802002224694
0.967741935483871
0.9733036707452726
0.978865406006674
0.9844271412680756
0.9899888765294772
0.9955506117908788
0.0
0.0055617352614015575
0.011123470522803115
0.01668520578420467
0.02224694104560623
0.027808676307007785
0.03337041156840934
0.0389321468298109
0.04449388209121246
0.05005561735261402
0.05561735261401557
0.06117908787541713
0.06674082313681869
0.07230255839822025
0.0778642936596218
0.08342602892102335
0.08898776418242492
0.09454949944382647
0.10011123470522804
0.10567296996662959
0.11123470522803114
0.1167964404894327
0.12235817575083426
0.12791991101223582
0.133481

In [41]:
if(False):#check for multicolineraity as a result of smoothing 4xn representation
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns

    df = pd.DataFrame()

    up = two_d["RPS_C_13_C3_left_head"][0,2000:3000]
    left = two_d["RPS_C_13_C3_left_head"][1,2000:3000]
    down = two_d["RPS_C_13_C3_left_head"][2,2000:3000]
    right = two_d["right"]=two_d["RPS_C_13_C3_left_head"][3,2000:3000]
    
    df["up"]=up
    df["left"]=left
    df["down"]=down
    df["right"]=right
    
    if(False):
        fig = plt.figure(figsize=(16,9))
        plt.plot(up)
        #plt.plot(left)
        plt.plot(down)
        #plt.plot(right)
        plt.plot(up*down/5)
    else:
        sns.heatmap(data=df.corr().round(2), annot=True)

In [4]:
with open(outfolder+"1d_movement.motion","wb") as f:
    pickle.dump(one_d,f)

In [8]:
with open(outfolder+"2d_movement.motion","wb") as f:
    pickle.dump(two_d,f)

In [6]:
# Test loading
with open(outfolder+"1d_movement.motion","rb") as f:
    one_d = pickle.load(f)

# Compute contrast between all electrodes in alpha, beta and gamma range

In [44]:
import os
import re
import numpy as np
from scipy import ndimage

import sys
import pickle
import json
import tables

import importlib.util #import from superordinate folder
spec = importlib.util.spec_from_file_location("helper", 
                                              str(re.search("(.*/)",os.getcwd()).groups(0)[0])+"helper.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

In [45]:
folder = "/mnt/c/Users/eler/Documents/Cognitive_Science/Bachelorarbeit/Scripts/processing_pipeline/dyad_13/"
eeg_path = folder+"DualEEG_RPS_C_13.eeg"
vmrk = folder+"DualEEG_RPS_C_13.vmrk"

### Participant 1

In [46]:
data = helper.frequency_bands_between_electrodes(path=eeg_path, participant = 0, vmrk =vmrk)

.........0.001953125
.........0.00390625
.........0.005859375
.........0.0078125
.........0.009765625
.........0.01171875
.........0.013671875
.........0.015625
.........0.017578125
.........0.01953125
.........0.021484375
.........0.0234375
.........0.025390625
.........0.02734375
.........0.029296875
.........0.03125
.........0.033203125
.........0.03515625
.........0.037109375
.........0.0390625
.........0.041015625
.........0.04296875
.........0.044921875
.........0.046875
.........0.048828125
.........0.05078125
.........0.052734375
.........0.0546875
.........0.056640625
.........0.05859375
.........0.060546875
.........0.0625
.........0.064453125
.........0.06640625
.........0.068359375
.........0.0703125
.........0.072265625
.........0.07421875
.........0.076171875
.........0.078125
.........0.080078125
.........0.08203125
.........0.083984375
.........0.0859375
.........0.087890625
.........0.08984375
.........0.091796875
.........0.09375
.........0.095703125
.........0.097656

In [13]:
with open(folder+"RPS_C_13_C3_left.bands","rb") as f:#test loading
    gamma_low_contrasts = np.load(f)

In [6]:
#data = helper.frequency_bands_between_electrodes(path=path, participant = 1, vmrk =vmrk)
with open(folder+"RPS_C_13_C3_left.bands","wb") as f:
    np.save(f, data)
    
df.style.applymap(color_range)

### Participant 2

In [5]:
data = helper.frequency_bands_between_electrodes(path=eeg_path, participant = 1, vmrk =vmrk)
with open(folder+"RPS_C_13_C3_right.bands","wb") as f:
    np.save(f, data)

.........0.001953125
.........0.00390625
.........0.005859375
.........0.0078125
.........0.009765625
.........0.01171875
.........0.013671875
.........0.015625
.........0.017578125
.........0.01953125
.........0.021484375
.........0.0234375
.........0.025390625
.........0.02734375
.........0.029296875
.........0.03125
.........0.033203125
.........0.03515625
.........0.037109375
.........0.0390625
.........0.041015625
.........0.04296875
.........0.044921875
.........0.046875
.........0.048828125
.........0.05078125
.........0.052734375
.........0.0546875
.........0.056640625
.........0.05859375
.........0.060546875
.........0.0625
.........0.064453125
.........0.06640625
.........0.068359375
.........0.0703125
.........0.072265625
.........0.07421875
.........0.076171875
.........0.078125
.........0.080078125
.........0.08203125
.........0.083984375
.........0.0859375
.........0.087890625
.........0.08984375
.........0.091796875
.........0.09375
.........0.095703125
.........0.097656